## Dataset

In [2]:
# Import standard libraries that you may use most times
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
mpl.rcParams['figure.figsize'] = (16, 5)
mpl.rcParams['axes.grid'] = False
# -------------------------------------------------------------------------
# 1. Some book keeping    
print("TensorFlow version:", tf.__version__)

2023-01-23 16:16:18.109460: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.11.0


In [83]:
from tensorflow.data import Dataset

In [84]:
ds = Dataset.range(10)
print([v.numpy() for v in ds])
ds1 = ds.window(5, shift=1)
print("Shift by 1:", [[v1.numpy() for v1 in v] for v in ds1])

ds1 = ds.window(5, shift=1, drop_remainder=True)
print("Shift by 2:", [[v1.numpy() for v1 in v] for v in ds1])


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Shift by 1: [[0, 1, 2, 3, 4], [1, 2, 3, 4, 5], [2, 3, 4, 5, 6], [3, 4, 5, 6, 7], [4, 5, 6, 7, 8], [5, 6, 7, 8, 9], [6, 7, 8, 9], [7, 8, 9], [8, 9], [9]]
Shift by 2: [[0, 1, 2, 3, 4], [1, 2, 3, 4, 5], [2, 3, 4, 5, 6], [3, 4, 5, 6, 7], [4, 5, 6, 7, 8], [5, 6, 7, 8, 9]]


2023-01-22 19:36:45.738002: W tensorflow/core/framework/dataset.cc:769] Input of Window will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


In [85]:
ds = Dataset.range(10)
ds1 = ds.window(5, shift=1, drop_remainder=True)
ds1 = ds1.flat_map(lambda w: w.batch(5))
for w in ds1:
    print(w, w.numpy())


tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int64) [0 1 2 3 4]
tf.Tensor([1 2 3 4 5], shape=(5,), dtype=int64) [1 2 3 4 5]
tf.Tensor([2 3 4 5 6], shape=(5,), dtype=int64) [2 3 4 5 6]
tf.Tensor([3 4 5 6 7], shape=(5,), dtype=int64) [3 4 5 6 7]
tf.Tensor([4 5 6 7 8], shape=(5,), dtype=int64) [4 5 6 7 8]
tf.Tensor([5 6 7 8 9], shape=(5,), dtype=int64) [5 6 7 8 9]


### Preparing Data for RNN LSTM

In [86]:
def print_dataset(ds):
    for inputs, targets in ds:
        print("---Batch---")
        print("Feature:", inputs.numpy())
        print("Label:", targets.numpy())
        print("")
 

In [132]:
def window(dataset, label_slice, input_sequence_length, output_sequence_length, batch_size):
    ds = dataset.window(input_sequence_length + output_sequence_length, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda x: x).batch(input_sequence_length + output_sequence_length)
     
    def split_feature_label(x):
        return x[:input_sequence_length], x[input_sequence_length:,label_slice]
     
    ds = ds.map(split_feature_label)
     
    return ds.batch(batch_size)
 
#ds = tf.data.Dataset.from_tensor_slices(simple_data_samples)
#ds = timeseries_dataset_from_dataset(ds, slice(3, None, None), input_sequence_length=4, output_sequence_length=2, batch_size=2)
#print_dataset(ds)

In [134]:
ds = tf.data.Dataset.from_tensor_slices(values)
ds = window(ds, slice(3, 5, None), input_sequence_length=4, output_sequence_length=2, batch_size=2)
print_dataset(ds)

---Batch---
Feature: [[[  1  10 100  -1  -1]
  [  2  20 200  -2  -2]
  [  3  30 300  -3  -3]
  [  4  40 400  -4  -4]]

 [[  2  20 200  -2  -2]
  [  3  30 300  -3  -3]
  [  4  40 400  -4  -4]
  [  5  50 500  -5  -5]]]
Label: [[[-5 -5]
  [-6 -6]]

 [[-6 -6]
  [-7 -7]]]

---Batch---
Feature: [[[  3  30 300  -3  -3]
  [  4  40 400  -4  -4]
  [  5  50 500  -5  -5]
  [  6  60 600  -6  -6]]

 [[  4  40 400  -4  -4]
  [  5  50 500  -5  -5]
  [  6  60 600  -6  -6]
  [  7  70 700  -7  -7]]]
Label: [[[-7 -7]
  [-8 -8]]

 [[-8 -8]
  [-9 -9]]]

---Batch---
Feature: [[[  5  50 500  -5  -5]
  [  6  60 600  -6  -6]
  [  7  70 700  -7  -7]
  [  8  80 800  -8  -8]]

 [[  6  60 600  -6  -6]
  [  7  70 700  -7  -7]
  [  8  80 800  -8  -8]
  [  9  90 900  -9  -9]]]
Label: [[[ -9  -9]
  [-10 -10]]

 [[-10 -10]
  [-11 -11]]]

---Batch---
Feature: [[[   7   70  700   -7   -7]
  [   8   80  800   -8   -8]
  [   9   90  900   -9   -9]
  [  10  100 1000  -10  -10]]]
Label: [[[-11 -11]
  [-12 -12]]]



In [88]:
#ds = tf.data.experimental.CsvDataset("../data/stockdata.csv", header=True) #, record_defaults=[5])
#ds = ds.map(lambda *items: tf.stack(items))
#ds = window(ds, slice(3, None, None), input_sequence_length=4, output_sequence_length=2, batch_size=2)
#print_dataset(ds)

## Loading Data using Pandas

In [4]:
file = '../data/jena_climate_2009_2016.csv.zip'
#file = '../data/stockdata.csv'
df = pd.read_csv(file)
df['Date Time'] = pd.to_datetime( df['Date Time'] )
df

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,2009-01-01 00:10:00,996.52,-8.02,265.40,-8.90,93.30,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
1,2009-01-01 00:20:00,996.57,-8.41,265.01,-9.28,93.40,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2,2009-01-01 00:30:00,996.53,-8.51,264.91,-9.31,93.90,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
3,2009-01-01 00:40:00,996.51,-8.31,265.12,-9.07,94.20,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
4,2009-01-01 00:50:00,996.51,-8.27,265.15,-9.04,94.10,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420546,2016-12-31 23:20:00,1000.07,-4.05,269.10,-8.13,73.10,4.52,3.30,1.22,2.06,3.30,1292.98,0.67,1.52,240.0
420547,2016-12-31 23:30:00,999.93,-3.35,269.81,-8.06,69.71,4.77,3.32,1.44,2.07,3.32,1289.44,1.14,1.92,234.3
420548,2016-12-31 23:40:00,999.82,-3.16,270.01,-8.21,67.91,4.84,3.28,1.55,2.05,3.28,1288.39,1.08,2.00,215.2
420549,2016-12-31 23:50:00,999.81,-4.23,268.94,-8.53,71.80,4.46,3.20,1.26,1.99,3.20,1293.56,1.49,2.16,225.8


In [124]:
def testvalues():
    values = np.array([
         [1, 10, 100,  -1, -1],
         [2, 20, 200,  -2, -2],
         [3, 30, 300,  -3, -3],
         [4, 40, 400,  -4, -4],
         [5, 50, 500,  -5, -5],
         [6, 60, 600,  -6, -6],
         [7, 70, 700,  -7, -7],
         [8, 80, 800,  -8, -8],
         [9, 90, 900,  -9, -9],
         [10,100,1000, -10, -10],
         [11,110,1100, -11, -11],
         [12,120,1200, -12, -12],
    ])
    return values;

def print_test(ds, len=4):
    for inputs, targets in ds:
        print(inputs.numpy() , targets.numpy())

In [125]:
window_len = 5
fcast_len  = 2
batch_size = 2

#values = df.values[:, 1:5]
values = testvalues()
dataset = tf.data.Dataset.from_tensor_slices(values)
dataset = dataset.window(window_len + fcast_len, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda x: x).batch(window_len + fcast_len)

def split_feature_label(x):
    return x[:window_len], x[window_len:,fcast_len]
     
dataset = dataset.map(split_feature_label)
dataset.batch(batch_size)
print_test(dataset)

[[  1  10 100  -1  -1]
 [  2  20 200  -2  -2]
 [  3  30 300  -3  -3]
 [  4  40 400  -4  -4]
 [  5  50 500  -5  -5]] [600 700]
[[  2  20 200  -2  -2]
 [  3  30 300  -3  -3]
 [  4  40 400  -4  -4]
 [  5  50 500  -5  -5]
 [  6  60 600  -6  -6]] [700 800]
[[  3  30 300  -3  -3]
 [  4  40 400  -4  -4]
 [  5  50 500  -5  -5]
 [  6  60 600  -6  -6]
 [  7  70 700  -7  -7]] [800 900]
[[  4  40 400  -4  -4]
 [  5  50 500  -5  -5]
 [  6  60 600  -6  -6]
 [  7  70 700  -7  -7]
 [  8  80 800  -8  -8]] [ 900 1000]
[[  5  50 500  -5  -5]
 [  6  60 600  -6  -6]
 [  7  70 700  -7  -7]
 [  8  80 800  -8  -8]
 [  9  90 900  -9  -9]] [1000 1100]
[[   6   60  600   -6   -6]
 [   7   70  700   -7   -7]
 [   8   80  800   -8   -8]
 [   9   90  900   -9   -9]
 [  10  100 1000  -10  -10]] [1100 1200]


In [131]:
vv=np.array([
 [ 1 , 1 , 1 ,-1 , -1],
 [ 2 , 2 , 2 ,-2 , -2],
 [ 3 , 3 , 3 ,-3 , -3],
 [ 4 , 4 , 4 ,-4 , -4],
 [ 5 , 5 , 5 ,-5 , -5],
 [ 6 , 6 , 6 ,-6 , -6],
 [ 7 , 7 , 7 ,-7 , -7]])

vv = values
vv[:5], vv[5:7,2:4]

(array([[  1,  10, 100,  -1,  -1],
        [  2,  20, 200,  -2,  -2],
        [  3,  30, 300,  -3,  -3],
        [  4,  40, 400,  -4,  -4],
        [  5,  50, 500,  -5,  -5]]),
 array([[600,  -6],
        [700,  -7]]))

In [112]:
values = testvalues()
dataset = tf.data.Dataset.from_tensor_slices(values)
dataset = dataset.window(window_len + fcast_len, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda x: x).batch(window_len + fcast_len)
for w in (dataset):
    print(w)

tf.Tensor(
[[ 1  1  1 -1 -1]
 [ 2  2  2 -2 -2]
 [ 3  3  3 -3 -3]
 [ 4  4  4 -4 -4]
 [ 5  5  5 -5 -5]
 [ 6  6  6 -6 -6]
 [ 7  7  7 -7 -7]], shape=(7, 5), dtype=int64)
tf.Tensor(
[[ 2  2  2 -2 -2]
 [ 3  3  3 -3 -3]
 [ 4  4  4 -4 -4]
 [ 5  5  5 -5 -5]
 [ 6  6  6 -6 -6]
 [ 7  7  7 -7 -7]
 [ 8  8  8 -8 -8]], shape=(7, 5), dtype=int64)
tf.Tensor(
[[ 3  3  3 -3 -3]
 [ 4  4  4 -4 -4]
 [ 5  5  5 -5 -5]
 [ 6  6  6 -6 -6]
 [ 7  7  7 -7 -7]
 [ 8  8  8 -8 -8]
 [ 9  9  9 -9 -9]], shape=(7, 5), dtype=int64)
tf.Tensor(
[[  4   4   4  -4  -4]
 [  5   5   5  -5  -5]
 [  6   6   6  -6  -6]
 [  7   7   7  -7  -7]
 [  8   8   8  -8  -8]
 [  9   9   9  -9  -9]
 [ 10  10  10 -10 -10]], shape=(7, 5), dtype=int64)
tf.Tensor(
[[  5   5   5  -5  -5]
 [  6   6   6  -6  -6]
 [  7   7   7  -7  -7]
 [  8   8   8  -8  -8]
 [  9   9   9  -9  -9]
 [ 10  10  10 -10 -10]
 [ 11  11  11 -11 -11]], shape=(7, 5), dtype=int64)
tf.Tensor(
[[  6   6   6  -6  -6]
 [  7   7   7  -7  -7]
 [  8   8   8  -8  -8]
 [  9   9   9  -9  -

In [ ]:
# Time series generator Example:

from keras.preprocessing.sequence import TimeseriesGenerator
dftt = pd.read_csv("");
gen = TimeseriesGenerator(dftt.values,dftt.values, length =window_len, batch_size = batch_size)
print("========================")
for g in gen:
    print(g[1])
    break;
